In [2]:
import numpy as np

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import row, column

from numpy.linalg import solve, cond, inv, det, eig, qr
from scipy.linalg import lu_factor, lu_solve, solve_banded

import time

output_notebook(hide_banner=True)

# Conditionnement

Considérons l'exemple suivant :
$$
    A=\begin{pmatrix}
      10 & 1 & 4 & 0 \\
      1  & 10& 5 & -1\\
      4 & 5 & 10 & 7 \\
      0 & -1 & 7 & 9
    \end{pmatrix}
    \text{ avec }
    b_1 = \begin{pmatrix}
    15\\15\\26\\15
    \end{pmatrix}
    \text{ et }
    b_2 = \begin{pmatrix}
    16\\16\\25\\16
    \end{pmatrix}
$$

**Calculez le déterminant de $A$ et l'inverse de $A$ avec les fonctions *numpy.linalg.inv* et *numpy.linalg.det***

In [3]:
A = np.array([[10,1,4,0],[1,10,5,-1],[4,5,10,7],[0,-1,7,9]])
b1 = np.array([15,15,26,15])
b2 = np.array([16,16,25,16])

detA = np.linalg.det(A)
invA = np.linalg.inv(A)

print ("le déterminant de A est : ",detA)
print ("l'inverse de A est : \n",invA)

le déterminant de A est :  1.0000000000010525
l'inverse de A est : 
 [[ 105.  167. -304.  255.]
 [ 167.  266. -484.  406.]
 [-304. -484.  881. -739.]
 [ 255.  406. -739.  620.]]


**Commentaire :**  
$A$ est une matrice à coefficients entiers, son déterminant est donc aussi un entier, sa valeur vaut 1. 
Le determinant obtenu par la fonction numpy.linalg.det nous a donné alors un résultat avec une erreur de $10^{-12}$, ce qui nous donne une première idée de la précision des fonction de la bibliothèque linalg.

**Résolvez les systèmes $Ax_1=b_1$ et $Ax_2 = b_2$ en utilisant soit l'inverse trouvée ci-dessus, soit *numpy.linalg.solve*. Expliquez le comportement obtenu et donnez une borne inférieure pour le conditionnement de la matrice $A$.**

In [4]:
# résolution en utilisant np.linalg.solve
x1 = np.linalg.solve(A,b1)
x2 = np.linalg.solve(A,b2)

# résolution en multipliant invA par bi. i=1,2
x1inv = np.dot(invA,b1)
x2inv = np.dot(invA,b2)

print("résolution en utilisant np.linalg.solve")
print("La solution de Ax = b1 est ",x1)
print("La solution de Ax = b2 est ",x2,"\n")
print("résolution en multipliant invA par bi")
print("La solution de Ax = b1 est ",x1)
print("La solution de Ax = b2 est ",x2)

résolution en utilisant np.linalg.solve
La solution de Ax = b1 est  [1. 1. 1. 1.]
La solution de Ax = b2 est  [  832.  1324. -2407.  2021.] 

résolution en multipliant invA par bi
La solution de Ax = b1 est  [1. 1. 1. 1.]
La solution de Ax = b2 est  [  832.  1324. -2407.  2021.]


**Commentaire :** 
Pour mieux analyser ce résultat, on va calculer les erreurs relatives $\frac{||\delta(x)||}{||x_1||}$ et $\frac{||\delta(b)||}{||b_1||}$

In [5]:
def norme2(x) :
    N2 = 0
    for i in range (len(x)) :
        N2 += x[i]**2
    return np.sqrt(N2)

In [6]:
dx = x2 - x1
db = b2 - b1

err_x = norme2(dx)/norme2(x1)
err_b = norme2(db)/norme2(b1)

print("erreur relative de la norme de x : ",err_x)
print("erreur relative de la norme de b : ",err_b)
print("rapport des deux erreurs : ", err_x/err_b)

erreur relative de la norme de x :  1754.9753559509859
erreur relative de la norme de b :  0.05441295617909432
rapport des deux erreurs :  32252.8948836133


**Commentaire :**  
On remarque que l'erreur relative de la norme de $b$ est très petite, on pouvait le prédire rien qu'en regardant les coefficients rapprochés de $b_1$ et $b_2$. Pourtant, l'erreur relative de la norme de $x$ est très grande par rapport à celle de $b$.
En s'appuyant sur le résultat de la question 1/ on peut expliquer ce résultat par le fait que le conditionnement de $A$ est très grand, car on a $\frac{||\delta(x)|| / ||x_1||}{||\delta(b)|| / ||b_1||} \leq \kappa(A)$.

Cette inégalité nous donne la majoration : $\kappa(A) \geq 32252.8948836133$

**Calculez le conditionnement de $A$ par les deux méthodes suivantes :**
* **calculez les valeurs propres de $A$ avec *numpy.linalg.eig* et utilisez le point précédent**
* **utiliser *numpy.linalg.cond***


In [7]:
vp = np.linalg.eig(A)
vp_abs = abs(vp[0])

# Recherche des valeurs propres max et min en valeur absolue
vp_max = vp_min = vp_abs[0]
for v in vp_abs :
    vp_max = max(v, vp_max)
    vp_min = min(v, vp_min)

# Calcul du conditionnement de A par les 2 méthodes
condA_eig = vp_max/vp_min
condA_cond = np.linalg.cond(A)

print("Le conditionnement obtenu à partir des valeurs propres de A est     : ",condA_eig)
print("Le conditionnement obtenu par la fonction numpy.linalg.cond à A est : ",condA_cond)
print("L'erreur entre les conditionnements calculés par deux méthodes est : ",abs(condA_eig - condA_cond))

Le conditionnement obtenu à partir des valeurs propres de A est     :  35792.39762891995
Le conditionnement obtenu par la fonction numpy.linalg.cond à A est :  35792.39762876955
L'erreur entre les conditionnements calculés par deux méthodes est :  1.504013198427856e-07


**Commentaire :** Les deux méthodes aboutissent à la même valeur numérique du conditionnement de $A$ avec une erreur de l'ordre de $10^{-7}$, et cette valeur vérifie l'inégalité établie précédemment : $\kappa(A) \geq 32252.8948836133$

# Décomposition QR et résolution de systèmes surdimensionnés

On considère le système surdimensionné $Ax=b$, avec 
$$
    A=\begin{pmatrix}
      10 & 1 & 4  \\
      1  & 3 & 5 \\
      4 & 5  & 10 \\
      2 & 5  & 6 \\
      1 & 8  & 6 \\
      5 & 9  & -4 \\
      0 & -1 & 7 
    \end{pmatrix}\ 
    \text{ et }\ 
    b = \begin{pmatrix}
    14\\15\\26\\28\\47\\70 \\-15
    \end{pmatrix}
$$

**Utiliser la méthode de la question du sujet de PC pour résoudre le système linéaire surdimensionné. On pourra utiliser la fonction *numpy.linalg.qr*.**

**Etude théorique :**  

Pour tout vecteur $x$ de taille $n$, on a :
$$||Ax-b||_2 = ||QRx-b||_2 = ||Rx - Q^T b||_2$$
$R$ est de la forme 
\begin{pmatrix}
      R_1  \\
      0_{m-n,n}
    \end{pmatrix}
où $R_1$ est une matrice triangulaire de taille n. Ainsi :
$$Rx - Q^T b = \begin{pmatrix}
              R_1 x - (Q^Tb)[:n]  \\
              -(Q^Tb)[n:]
                \end{pmatrix}$$
Et donc $$\quad ||Rx - Q^T b||_2^2 \quad = \quad ||\, \,R_1x - (Q^T b)[:n] \, \,||_2^2 \quad + \quad ||\, \,(Q^Tb)[n:] \, \,||_2^2$$

Minimiser $||Ax-b||_2$ revient donc à minimiser $||\, \,R_1x - (Q^T b)[:n] \, \,||_2$.  
$R_1$ est triangulaire à coefficients diagonaux non nuls, elle est donc inversible. Le vecteur solution de notre problème est alors la solution du système linéaire $R_1x = (Q^T b)[:n]$.

In [11]:
def sol_min_qr(A,b) :
    # Décomposition QR de A
    qrA = np.linalg.qr(A, mode = "complete")
    Q, R = qrA[0], qrA[1]
    
    # R1, b1 
    n = len(R[0])
    R1 = R[:n]
    b1 = np.dot(Q.T,b)[:n]
    
    # Solution de R1.x = b1
    return np.linalg.solve(R1,b1)

In [12]:
# Application au système surdimmentionné
A=np.array([[10,1,4],[1,3,5],[4,5,10],[2,5,6],[1,8,6],[5,9,-4],[0,-1,7]])
b=np.array([14,15,26,28,47,70,-15])

x = sol_min_qr(A,b)
print ("Le vecteur minimisant l'erreur ||Ax-b|| est x = ",x)

Le vecteur minimisant l'erreur ||Ax-b|| est x =  [ 1.210814    6.58773664 -1.18569127]


# Stockage de matrice et Laplacien

 ## Cas 1D

 ### Matrice pleine
 
On considère une matrice $A$ tridiagonale, *i.e.* dont les seuls coefficients non-nuls sont donnés par :
$A_{i,j} \neq 0 \quad\Rightarrow\quad |i-j| \le 1.$  

On écrit cette matrice $A$ sous la forme
  $$ A = \left( \begin{array}{cccccc}
    a_1    & b_1    & 0       &\dots   & 0 \\
    c_2    & a_2    & b_2     & \ddots& \vdots  \\
    0      & \ddots & \ddots &\ddots  &0\\
    \vdots & \ddots & \ddots &\ddots  & b_{N-1}\\
    0      & \dots  &  0   &   c_N  & a_N
  \end{array}\right).$$
  
  
Pour les applications numériques, on fixera :
\begin{equation} a_i = \frac{-2}{h^2}, \quad b_i = \frac{1}{h^2} \quad \text{ et } c_i = \frac{1}{h^2} \text{ avec } h = \frac{1}{N+1}, \quad N = 2048.
\end{equation}

Ici $h$ est un pas d'espace et $N$ un nombre de mailles. Cette matrice correspond à une discrétisation de l'opérateur $-\Delta$, dont la construction sera vue plus tard dans le cours.

**On utilisera les valeurs numériques ci-dessus. Utiliser la fonction *scipy.linalg.lu_factor* pour construire deux matrices $L$ et $U$ respectivement triangulaire inférieure et supérieure telles que $A=LU$.**

La fonction scipy.linalg.lu_factor retourne une matrice dont la partie strictement supérieure à la diagonale contient les élements de L, et la partie inférieure à la diagonale contient les élements de U (les élements diagonaux de L sont des 1). On construira L et U avec la fonction decompLU.

In [13]:
# decompLU retourne les matrices P (de permutation), L et U correspondant à la décomposition LU de A : A = P*L*U,

def decompLU(A) :
    N = len(A)
    LU, piv = lu_factor(A)
    
    # Construction de L
    L = np.eye(N)
    for i in range(1,N) :
        L[i][i-1] = LU[i][i-1]

    # Construction de U
    U = np.zeros((N,N))
    for i in range(N) :
        for j in range(i,N) :
            U[i][j] = LU[i][j]
    
    # Construction de P
    P = np.zeros((N,N))
    for i in range(N) :
        P[i][piv[i]] = 1
    
    return P, L, U

In [14]:
# Le choix de N=2048 fait dans l'énoncé vise à manipuler des puissances de 2 dans notre programme,
# mais comme h=(N+1), c'est plutot N+1 qui devrait être une puissance de 2.
N = 2048-1

# Construction de A
h = 1/(N+1)
b = (N+1)**2   # =1/h**2
a = -2*b
A = np.array(a*np.eye(N))
for i in range(1,N) :
    A[i][i-1] = A[i-1][i] = b

# ti, tf vont nous servir à calculer le temps d'excecution, le résultat sera utilisé dans la question 2/f)
ti = time.time()

# Calcul de la décomposition Lu de A
P, L, U = decompLU(A)
tf = time.time()

## Pour vérifier le résultat obtenu, on peut efffectuer les operations ci-dessous, mais qui sont couteuses
"""
A1 =  np.dot(P,np.dot(L,U))
#print("La matrice A :\n", A)
#print("La matrice P*L*U : \n", A1)
"""

print("Les deux matrices A et P*L*U sont identiques")
print("Le temps total de calcul de la décomosition LU de A est ",tf-ti,"s")

Les deux matrices A et P*L*U sont identiques
Le temps total de calcul de la décomosition LU de A est  6.740319490432739 s


**Résoudre $-Au=f$ avec $f_i = x_i^3$ où $x_i = ih$ en utilisant la factorisation LU précédente et la fonction *scipy.linalg.lu_solve*.**

In [16]:
xi = h*np.array([i for i in range(1,N+1)])
f =  xi*xi*xi

# Résolution du système Au = -f :
ti = time.time()
LU, piv = lu_factor(A)
ui = lu_solve((LU,piv), -f)
tf = time.time()

print("le temps total de la résolution du système A.u = -f est ",tf-ti,"s")
print("La solution du système A.u=-f est représentée ci-dessous par la courbe reliant les points (xi,ui)")

le temps total de la résolution du système A.u = -f est  1.7296969890594482 s
La solution du système A.u=-f est représentée ci-dessous par la courbe reliant les points (xi,ui)


**Calculer la solution analytique $x\mapsto u(x)$ du problème $-\Delta u = f$ et tracer les courbes $u(x_i)$ et $u_i$ en fonction de $x_i$.**

Calculons la solution analytique de ce problème :  
$\forall x \in [0,1], \Delta u = -x^3$ donc, en integrant deux fois on trouve $ \forall x \in [0,1], u(x) = -x^5/20 + \alpha x +\beta$  
Les conditions $u(0) = u(1) = 0$ impliquent que $\forall x \in [0,1] : u(x) = (x-x^5)/20 = x(1-x)(1+x)(1+x^2)/20 $.

In [17]:
# Image de xi par u
def u(x) : 
    return x*(1-x)*(1+x)*(1+x*x)/20
yi = u(xi)

# Erreur
err = abs(ui-yi)

# Figure
fig = figure(width=490, height=300)
fig.line(xi,ui, legend = "ui")
fig.line(xi,yi, color = "orange", legend = "u(xi)")
fig.legend.location = "top_left"

fig2 = figure(width=490, height=300, x_axis_type="log", y_axis_type="log")
fig2.line(xi, err, legend ="|u(xi)-ui|")
fig2.legend.location = "top_left"

show(row(fig,fig2))


**Commentaire :** Les deux courbes $(x_i,u_i)$ et $(x_i, u(x_i))$ sont identiques à une erreur près qui ne dépasse jamais $10^{-8}$

### Matrice tridiagonale

**Implémenter l'algorithme de décomposition LU basé sur un stockage tridiagonal de la matrice, c'est-à-dire qu'on ne stocke que les composantes non-triviales des matrice $A$, $L$ et $U$. On écrira pour cela :**

\begin{gather}
      \tilde{A} = \left( \begin{array}{c|c|c|c} 0 & b_{1} & \dots & b_{N-1}\\ \hline  a_{1} & a_{2} & \dots & a_{N} \\ \hline c_{2} & c_{3} & \dots  & 0 \end{array} \right) = \left( \begin{array}{l} b \\ a \\ c \end{array}\right) \in \mathbb{R}^{3,N}, \nonumber \\
      \tilde{L} = \left( \gamma_{2}, \gamma_{3}, \dots, \gamma_{N}, 0 \right) \in\mathbb{R}^{1,N}, \qquad
      \tilde{U} = \left( \begin{array}{c|c|c|c|c} 0 & \beta_2 & \beta_3 & \dots & \beta_N \\ \hline \alpha_{1} & \alpha_{2} & \alpha_{3} & \dots & \alpha_{N} \end{array} \right) \in\mathbb{R}^{2,N}.
      %\tilde{A} = \left( \begin{array}{c|c|c} 0 & A_{1,1} & A_{1,2} \\ A_{2,1} & A_{2,2} & A_{2,3} \\ \vdots & \vdots & \vdots \\  A_{N,N-1} & A_{N,N} & 0 \end{array} \right) \in \mathbb{R}^{3,N}, \nonumber \\
      %\tilde{L} = \left( \begin{array}{c} 0 \\ L_{2,1} \\ L_{3,2} \\ \vdots \\ L_{N,N-1}\end{array}\right) \in\mathbb{R}^{N,1}, \qquad
      %\tilde{U} = \left( \begin{array}{c|c} U_{1,1} & U_{1,2} \\ \vdots & \vdots \\ U_{N-1,N-1} & U_{N-1,N} \\ U_{N,N} & 0\end{array} \right) \in\mathbb{R}^{N,2}.
        %= (0, \ A_{1,1},\ A_{1,2}), \quad \tilde{A}_{1,i} = (A_{i,i-1}, \ A_{i,i}, \ A_{i,i+1}), \quad \tilde{A}_{N,:} = (A_{N,N-1},\ A_{N,N}, \ 0).
\end{gather}

**Commentaire :**  
En explicitant le produit $L.U$, et sachant que $A = L.U$, on obtient :
$$ a_1 = \alpha_1, \quad \forall i\geq 2,\quad a_i = \alpha_i + \gamma_i \beta_i,\quad c_i = \gamma_i \alpha_{i-1}, \quad b_{i-1} =\beta_i $$ 
Les coefficients $\alpha_i, \beta_i, \gamma_i$ peuvent donc être calculés récursivement comme suit :
$$\forall i\geq 2, \quad \beta_i = b_{i-1}$$
$$\alpha_1 = a_1, \quad \forall i\geq 2, \quad \gamma_i = c_i/\alpha_{i-1}, \quad \alpha_i = a_i - b_{i-1}\gamma_i$$ 

In [18]:
# stock_tridiag() prend en paramètre une matrice A de taille (N,N)
# et retourne une matrice de taille (3,N) correspont au stockage tridiagonal de A
def stock_tridiag(A) :
    a = np.array([A[i][i] for i in range(len(A))])
    b = np.array([0]+[A[i-1][i] for i in range(1,len(A))])
    c = np.array([A[i][i-1] for i in range(1,len(A))]+[0])
    return np.vstack((b,a,c))

# decompLU_tridiag() prend en argument un stockage tridiagonal At d'une matrice A
# et retourne le stockage tridiagonal de sa décompostion LU
def decompLU_tridiag(At) :
    b, a, c = At[0], At[1], At[2]
    N = len(b)
    alpha = np.zeros(N)
    gamma = np.zeros(N)
    alpha[0] = a[0]
    for i in range(1,N) :
        gamma[i-1] = c[i-1]/alpha[i-1]
        alpha[i] = a[i] - b[i]*gamma[i-1]
    Lt = gamma
    Ut = np.vstack((b, alpha))
    return Lt, Ut

**Tester votre algorithme sur la matrice :**

\begin{equation} 
A = \left(\begin{array}{ccc} 2 & 1 & 0 \\ -1 & 3 & 1 \\ 0 & 1 & 4 \end{array}\right).
\end{equation}

In [21]:
# Test de validité des fonctions stock_tridiag() et decompLU_tridiag()
A = np.array([[2,1,0],[-1,3,1],[0,1,4]])
P, L, U = decompLU(A)

At = stock_tridiag(A)
Lt, Ut = decompLU_tridiag(At)

print("La matrice pleine A : \n",A)
print("Stockage tridiagonal de A :\n " ,At)
print("La matrice pleine L : \n",L)
print("Stockage tridiagonal de L :\n " ,Lt)
print("La matrice pleine U : \n",U)
print("Stockage tridiagonal de U :\n " ,Ut)

La matrice pleine A : 
 [[ 2  1  0]
 [-1  3  1]
 [ 0  1  4]]
Stockage tridiagonal de A :
  [[ 0  1  1]
 [ 2  3  4]
 [-1  1  0]]
La matrice pleine L : 
 [[ 1.          0.          0.        ]
 [-0.5         1.          0.        ]
 [ 0.          0.28571429  1.        ]]
Stockage tridiagonal de L :
  [-0.5         0.28571429  0.        ]
La matrice pleine U : 
 [[2.         1.         0.        ]
 [0.         3.5        1.        ]
 [0.         0.         3.71428571]]
Stockage tridiagonal de U :
  [[0.         1.         1.        ]
 [2.         3.5        3.71428571]]


**Commentaire :** Les résultats obtenus sont corrects.

**Adapter et implémenter les algorithmes de remontée et de descente adaptés aux matrices triangulaires stockées sous forme tridiagonale. Tester votre algorithme pour la résolution de $A u = f$ avec la matrice $A$ de question précedente et $f = (1,\ 1,\ 1)^T$, en utilisant la décomposition $LU$ de $A$ obtenue à la question précédente.**

**Commentaire :**  
Ecrire $Au = -f$ revient à écrire $L(U.u) = -f$, notre problème se ramène donc à la résolution du problème 
$$ U.u = y, \quad L.y = -f$$
En explicitant les coefficients de $L$ et $U$, on obtient :  
$y_1 = f_1$  
$\forall i\geq 1 , \quad y_i = f_i - \gamma_i y_{i-1}$  
$u_N = y_N/\alpha_N$  
$\forall i \geq 1 , \quad u_{N-i} = (y_{N-i} - \beta_{N-i}u_{N-i+1})/\alpha_{N-i}$

In [22]:
# solve_L_descente() prend en paramètre Lt, correspondant au tridiagonal de L, et un vecteur f de même taille
# retourne la solution de L.y = f, calculée avec un algorithme de descente
def solve_L_descente(Lt,f) :
    y = np.zeros(len(f))
    y[0] = f[0]
    for i in range(1,len(f)) :
        y[i] = f[i] - Lt[i-1]*y[i-1]
    return y

# solve_U_remontee() prend en paramètre Ut, correspondant au tridiagonal de U, et un vecteur y
# retourne la solution de U.u = y, calculée avec un algorithme de remontée
def solve_U_remontee(Ut, y) :
    alpha, beta  = Ut[1], Ut[0]
    u = np.zeros(len(y))
    u[-1] = y[-1]/alpha[-1]
    for i in range(2,len(y)+1) :
        u[-i] = (y[-i] - beta[-i+1]*u[-i+1])/alpha[-i]
    return u

# solveLU_tridiag() prend en paramètre une matrice pleine A, et un vecteur f
# retourne la solution de A.u = f, calculée avec un algorithme de remontée et descente
def solveLU_tridiag(A,f) :
    A = stock_tridiag(A)
    Lt, Ut = decompLU_tridiag(A)
    y = solve_L_descente(Lt,f)
    u = solve_U_remontee(Ut, y)
    return u
    

In [25]:
f = np.array([1,1,1])
u = solveLU_tridiag(A,-f)
print("Problème : résoudre le système -Au=f, pour f = (1,1,1)")
print("Le vecteur solution obtenu avec l'algorithme de remontée et de déscente est u=",u)
print("Vérification du résultat : A*u = ",np.dot(A,u))

Problème : résoudre le système -Au=f, pour f = (1,1,1)
Le vecteur solution obtenu avec l'algorithme de remontée et de déscente est u= [-0.30769231 -0.38461538 -0.15384615]
Vérification du résultat : A*u =  [-1. -1. -1.]


**Commentaire :** J'ai choisi de donner à **solveLU_tridiag()** A comme paramètre et pas At, pour avoir les mêmes entrées (A,f) que l'algorithme de résolution utilisé dans la question 1/, ce qui nous permettra de comparer plus fiablement les deux algorithmes.

**Résoudre le système $-Au = f$ avec la matrice $A$ issue de la discrétisation de l'opérateur $-\Delta$ . Comparer la précision des résultats et les temps d'execution (en utilisant *time.time*) avec ceux obtenus avec la matrice pleine. Commenter les avantages et inconvénients de ce stockage.**

In [27]:
N = 2048-1

# Redéfinition de u
def u(x) : 
    return x*(1-x)*(1+x)*(1+x*x)/20

# Construction de A
h = 1/(N+1)
b = (N+1)**2   # =1/h**2
a = -2*b
A = np.array(a*np.eye(N))
for i in range(1,N) :
    A[i][i-1] = A[i-1][i] = b

# Construction de f
xi = h*np.array([i for i in range(1,N+1)])
f =  xi*xi*xi

# Résolution de A.u = -f en utilisant les matrices tridiagonales
tini = time.time()
ui = solveLU_tridiag(A,-f)
tfin = time.time()
print("Le temps de la résolution du système A.u = -f est ",tfin-tini,"s")

# Comparaison avec la solution analytique
yi = u(xi)
err = abs(ui-yi)

# Figure
fig = figure(width=490, height=300)
fig.line(xi,ui, legend = "ui")
fig.line(xi,yi, color = "orange", legend = "u(xi)")
fig.legend.location = "top_left"

fig2 = figure(width=490, height=300, x_axis_type="log", y_axis_type="log")
fig2.line(xi, err, legend ="|u(xi)-ui|")
fig2.legend.location = "top_left"

show(row(fig,fig2))

Le temps de la résolution du système A.u = -f est  0.03598284721374512 s


**Commentaire :**  
En terme de temps de calcul, le nouvel algorithme est plus performant que le premier : la résolution du système -A.u=f se fait en environ 0.04 s, alors que pour le premier algorithme, elle se fait en environ 1.70 s.  
En terme de précision, les deux algorithmes sont equivalents : la courbe d'erreur a la mçeme allure, et sa valeur maximale est de l'ordre de $10^{-8}$.

Le stockage tridiagonal permet alors d'optimiser le temps de calcul, et surtout l'espace de stockage : pour une matrice tridiagonale de taille (N,N) on va stocker les matrices At, Lt, et Ut de tailles respectives (3,N), (2,N) et (1,N), donc au total $6N$ valeurs. Alors qu'avec le premier algorithme, on stocke les matrices A, LU et piv de tailles respectives (N,N), (N,N) et (1,N), soit $2N^2+N$ valeurs.

## Cas 2D

On considère maintenant une matrice bande, *i.e.* dont les coefficients satisfont si $|i-j| < K$ alors$A_{i,j} = 0,$ où $K>0$ est appelée la largeur de bande.

  On utilisera pour les applications numériques la matrice $A$ définie par :
  
\begin{align}
    A &= \left( \begin{array}{c|c|c|c}
      D    & I/h^2    &\dots   & 0 \\ \hline
      I/h^2    & D    & \ddots& \vdots  \\ \hline
      \vdots   & \ddots &\ddots  &I/h^2\\ \hline
      0      & \dots  &   I/h^2  & D
    \end{array}\right), \quad %\in\mathbb{R}^{N^2\times N^2}, \
    D = \left( \begin{array}{cccc}
      -2/h^2    & 1/h^2    & \dots   & 0 \\
      1/h^2    & -2/h^2    & \ddots& \vdots  \\
      0      & \ddots & \ddots  &0\\
      0      & \dots  &  1/h^2  & -2/h^2
    \end{array}\right) \in\mathbb{R}^{N\times N},
    \label{Lapl_2D}
\end{align}

où $I$ est la matrice identité de taille $N=64$ et $h = \frac{1}{N}$. Cette matrice correspond à une discrétisation de l'opérateur $-\Delta$ en 2D.

### Matrices pleines

**En utilisant la fonction *solve* de numpy, résoudre le système $Au=f$ avec $f_{(i-1)N+j} = f(x_i,y_j) = x_i^3 + y_j^3$ où $x_i = (i-\frac{1}{2})h$ et $y_j = (j-\frac{1}{2})h$. Commenter la vitesse d'execution de l'algorithme et la taille de la matrice stockée (*sauvegarder avant d'executer*).**

In [89]:
N = 64-1
h = 1/(N+1)


# Construction de f
xi = yj = h*np.array([i for i in range(1,N+1)])
xi3 = xi*xi*xi #Pour éviter de recalculer à chaque fois xi[i]**3
yj3 = xi3
f = np.zeros(0)
for i in range(N) :
    f = np.hstack((f, xi3[i] + yj3))

tiA = time.time()
# Construction de D
invh2 = (1/h)**2 # pour éviter de recalculer à chaque fois l'inverse de h^2
D = np.zeros((N,N)) # 1/h = N+1
D[0][0] = -4*invh2
for i in range(1,N) :
    D[i][i-1] = D[i-1][i] = invh2
    D[i][i] = -4*invh2

## Construction de A ------------------------------------------------------------------------
I_h2 = invh2*np.eye(N)
Z = np.zeros((N,N))
# La première ligne de blocs de A
A = np.hstack((D,I_h2))
for i in range(2,N) :
    A = np.hstack((A,Z))

# Les lignes de blocs de A de la 2ème à la (N-1)ème
for l in range(1,N-1) :
    Al = np.zeros((N,0))
    for i in range(l-1) :
        Al = np.hstack((Al, Z))
    Al = np.hstack((Al, I_h2))
    Al = np.hstack((Al, D))
    Al = np.hstack((Al, I_h2))
    for i in range(l+2,N) :
        Al = np.hstack((Al, Z))
    A = np.vstack((A,Al))

# La dernière ligne de blocs de A
Al = np.zeros((N,0))
for i in range(N-2) :
    Al = np.hstack((Al, Z))
Al = np.hstack((Al, I_h2))
Al = np.hstack((Al, D))
A = np.vstack((A,Al))
#------------------------------------------------------------------------------------------
tfA = time.time()

ti = time.time()
u = np.linalg.solve(A,-f)
tf = time.time()

print(A)

print("Pour N = ",N," :")
print("le temps de construction de A est ",tfA-tiA,"s")
print("le temps de résolution du système A.u = -f est ",tf-ti,"s")

[[-64.  16.   0.  16.   0.   0.   0.   0.   0.]
 [ 16. -64.  16.   0.  16.   0.   0.   0.   0.]
 [  0.  16. -64.   0.   0.  16.   0.   0.   0.]
 [ 16.   0.   0. -64.  16.   0.  16.   0.   0.]
 [  0.  16.   0.  16. -64.  16.   0.  16.   0.]
 [  0.   0.  16.   0.  16. -64.   0.   0.  16.]
 [  0.   0.   0.  16.   0.   0. -64.  16.   0.]
 [  0.   0.   0.   0.  16.   0.  16. -64.  16.]
 [  0.   0.   0.   0.   0.  16.   0.  16. -64.]]
Pour N =  3  :
le temps de construction de A est  0.005995988845825195 s
le temps de résolution du système A.u = -f est  0.0009965896606445312 s


**Commentaire :**  
En faisant des tests avec différentes valeurs de $N$ ($N\leq 55$), on remarque que la construction de A et la résolution du système $Au=-f$ prennent un temps qui croît très rapidement avec $N$.  
Pour la valeur $N = 64$ le programme s'arrête et ne peut pas aller au bout du calcul. Ceci est dû au à la grande taille des opérations mais aussi à la grande taille de mémoire utilisée ($N^4$ valeurs stockées pour A) . Même la construction de A sans résolution du système bloque le programme.

### Matrices bandes

On stocke désormais la matrice sous forme bande, c'est-à-dire qu'on écrit :

\begin{align*}
      \tilde{A} = \left(
      \begin{array}{c|c|c|c|c|c}
        0             & \dots & 0        & A_{1,K+1} & \dots   & A_{N-K,N}\\
        \hline \vdots &       &  & A_{2,K+1} &\dots   & A_{N-K+1,N}\\
        \hline 0      & A_{1,2} &          &          &         & \vdots    \\
        \hline A_{1,1} & A_{2,2} & \dots  & \dots     & \dots   & A_{N,N} \\
        \hline A_{2,1} & A_{3,2} & \dots   & \dots    & A_{N,N-1} & 0 \\
        \hline \vdots &        &         &          &         & \vdots \\
        \hline A_{K+1,1} & \dots & A_{N,N-K} & 0       & \dots   &  0
      \end{array} \right) \in \mathbb{R}^{2K+1,N}.
\end{align*}

**Commentaire :**  
Dans notre cas, la taille de la matrice $A$ est $(N^2,N^2)$, $K = N$, les coefficients diagonaux valent $-4/h^2$, et les coefficients des 2èmes et $(N+1)$èmes diagonales (haut et bas) valent $1/h^2$.

In [115]:
N = 64-1
h = 1/(N+1)
invh2 = 1/(h*h)

ti = time.time()
# Construction de At ---------------------------------------
At = np.zeros((2*N+1,N*N))

# Diagonales n°1 et 2
At[N][0] = -4*invh2
for i in range(1,N*N) :
    At[N][i] = -4*invh2
    At[N-1][i] = At[N+1][i] = invh2

# Diagonales n°(N+1)
for i in range(N,N*N) :
    At[0][i] = At[2*N][i] = invh2
#------------------------------------------------------------
tf = time.time()

print("Le temps de construction de At est ",tf-ti,"s")

Le temps de construction de At est  0.023989200592041016 s


**Commentaire :**  
Alors que la construction de $A$ pour $N=64$ bloquait le programme, elle se fait à présent en mois de 0.03 secondes avec la nouvelle facon de stockage.  
Le nombre de valeurs stockées est $N^2(2N+1)$ à la place de $N^4$

**Utiliser la fonction *solve_banded* de scipy exploitant cette structure bande pour résoudre le problème $Au = f$. Comparer la vitesse d'execution et la taille de la matrice utilisée avec la question précédente.**

In [116]:
# Construction de f
xi = yj = h*np.array([i for i in range(1,N+1)])
xi3 = xi*xi*xi #Pour éviter de recalculer à chaque fois xi[i]**3
yj3 = xi3
f = np.zeros(0)
for i in range(N) :
    f = np.hstack((f, xi3[i] + yj3))


# Résolution du système A.u = -f
ti = time.time()
u = solve_banded((N,N),At,-f)
tf = time.time()

print("Le temps de résolution du système -Au = f est ",tf-ti,"s")

Le temps de résolution du système -Au = f est  0.08796191215515137 s


l'élément $u_{(i-1)N+j}$ doit donner une approximation de $u(x_i,y_j)$ 

**Commentaire :**  
Pour $N=64$, le temps de construction du stockage en bande de $A$ et du calcul de la solution de $-Au=f$ est de l'ordre de 0.1 s, alors que la construction de A sans résolution du système n'était pas faisable avec un stockage en matrice pleine de $A$, ce qui montre l'efficacité de cette méthode

### Matrices creuses CSR

**Bonus. Construire la matrice $A$ de la discrétisation de l'opérateur $-\Delta$ en format CSR (utiliser scipy.sparse.csr matrix) avec Scipy (https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html).**

**Bonus. Résoudre le système $Au = f$ avec la fonction *scipy.sparse.linalg.splu* et comparer la vitesse d’exécution et la taille de la matrice utilisée avec les deux méthodes précédentes.**